<a href="https://colab.research.google.com/github/jkimtokyo/LDA/blob/main/LDA_1URL_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 이것은 사본입니다.
!pip install konlpy
!pip install gensim
!pip install beautifulsoup4
!pip install requests

# SudachiPy와 관련된 라이브러리 설치 (일본어 형태소 분석을 위해)
!pip install sudachipy
!pip install sudachidict_core


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 7.7 MB/s eta 0:00:00


In [4]:
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from gensim import corpora, models
from sudachipy import tokenizer
from sudachipy import dictionary
from collections import defaultdict

# Corrected raw URLs for stopwords files
kr_stopword_url = 'https://raw.githubusercontent.com/your_github_username/your_repository/branch/kr-stopword.txt'
jp_stopword_url = 'https://github.com/jkimtokyo/LDA/blob/7c529a3deedf64964a6925c7119dc2cea12ce654/jp-stopword.txt'

# Function to download stopwords from the corrected URLs
def download_stopwords(url):
    response = requests.get(url)
    stopwords = response.text.split('\n')
    return [line.strip() for line in stopwords if line.strip()]

# Download stopwords using the corrected function
kr_stopwords = download_stopwords(kr_stopword_url)
jp_stopwords = download_stopwords(jp_stopword_url)

# 웹페이지 텍스트 추출 함수
def get_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text(separator=' ', strip=True)
    return text

# 형태소 분석 및 불용어 처리 함수
def analyze_text(lang, text, stopwords):
    processed_text = []
    if lang == 'kr':
        analyzer = Okt()
        tokens = analyzer.nouns(text)
    elif lang == 'jp':
        analyzer = dictionary.Dictionary().create()
        mode = tokenizer.Tokenizer.SplitMode.C
        tokens = [m.surface() for m in analyzer.tokenize(text, mode)]
    processed_text = [word for word in tokens if word not in stopwords and len(word) > 1]
    return processed_text

# LDA 모델 생성 및 결과 출력 함수, 주요 단어 15개 추출 및 합산
def create_lda_model(processed_docs, num_topics=12, num_words=24):
    dictionary = corpora.Dictionary(processed_docs)
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

    # 모든 토픽에 걸쳐 단어와 스코어를 합산
    word_scores = defaultdict(float)
    for idx in range(num_topics):
        for word, score in lda_model.show_topic(idx, topn=num_words):
            word_scores[word] += score

    # 합산된 스코어를 기준으로 내림차순 정렬하여 출력
    sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words:
        print(f"{word}: {score:.3f}")

# 메인 실행
if __name__ == "__main__":
    lang = input("Enter language (kr for Korean, jp for Japanese): ")
    url = input("Enter URL: ")
    text = get_text_from_url(url)
    stopwords = kr_stopwords if lang == 'kr' else jp_stopwords
    processed_text = analyze_text(lang, text, stopwords)
    processed_docs = [processed_text]
    create_lda_model(processed_docs, num_topics=12, num_words=24)


Enter language (kr for Korean, jp for Japanese): jp
Enter URL: https://ja.wikipedia.org/wiki/%E9%AB%98%E5%AE%97_(%E6%9C%9D%E9%AE%AE)
高宗: 0.026
朝鮮: 0.025
皇帝: 0.020
日本: 0.019
いる: 0.018
大院: 0.017
する: 0.017
編集: 0.015
国王: 0.014
閔妃: 0.014
ロシア: 0.014
こと: 0.013
大韓: 0.013
勢力: 0.012
この: 0.012
政策: 0.011
日報: 0.010
興宣: 0.010
世子: 0.010
韓国: 0.010
閔氏: 0.009
後宮: 0.008
帝国: 0.007
1947: 0.007
動き: 0.004
ある: 0.003
事件: 0.002
独立: 0.002
貴人: 0.002
1910: 0.002
金氏: 0.001
政治: 0.001
1897: 0.001
1919: 0.001
ため: 0.001
12: 0.001
哲宗: 0.001
記事: 0.001
ページ: 0.001
から: 0.001
指定: 0.001
より: 0.001
